# TA's NOTES FROM PHASE 2 SUBMISSION: Good work on webscraping. Just a few conversion of types and you are ready for some analysis! But don't forget to evaluate the quality and source of your data. Refer to the "data description" section of the final deliverable.

__To begin, libraries must be imported. There are some that are not part of the default python package.__  
Geckodriver and selenium are used, and firefox is used as the webdriver.

In [1]:
import time
import numpy as np
import pandas as pd

from selenium import webdriver #to get nhl pages after scripts load
from bs4 import BeautifulSoup
import requests

browser = webdriver.Firefox()
base_url = 'https://records.nhl.com/draft/draft-picks?league=all&team=all&year='
# page = requests.get(teams_url)
# soup = BeautifulSoup(page.content, 'html.parser')

__Function to get all first pick players from a specific year__

In [2]:
def add_year_draft_data(year):
    
    global draft_df
    
    year_url = base_url + str(year)
    browser.get(year_url)

    time.sleep(0.5)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    rows = soup.find_all('div', class_ = 'rt-tr-group')
    
    for row in rows:
        td = row.find_all('div', class_ = 'rt-td')
        if (td[3].text == '1'):
            dictionary = {
                'Name':td[1].text, 
                'Team':td[2].text, 
                'Pick':td[4].text, 
                'Position':td[6].text, 
                'Country':td[7].text, 
                'Height':td[8].text, 
                'Weight':td[9].text, 
                'Year':year    
            }
            link = td[1].find('a', href=True)
            if link != None:
                dict = add_individual_player_data(link['href'])
                try: 
                    dictionary.update(dict)
                except:
                    pass
            #draft_df = draft_df.append({'Name':td[1].text, 'Team':td[2].text, 'Pick':td[4].text, 'Position':td[6].text, 'Country':td[7].text, 'Height':td[8].text, 'Weight':td[9].text, 'Year':year}, ignore_index = True)
            draft_df = draft_df.append(dictionary, ignore_index=True)
            
            

In [3]:
def add_individual_player_data(link):
    try:
        browser.get(link)
        time.sleep(0.25)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        bio_list = soup.find_all('li', class_ = 'player-bio__item')
        temp = []
        for c in bio_list[3]:
            temp.append(c)
        shoots = temp[2].strip()



        stats_list = soup.find_all('tfoot')
        for element in stats_list:
            span = element.find_all('span')


        dict = {
            "Shoots": shoots,
            "Games-Played": span[2].text,
            "Goals": span[3].text,
            "Assists": span[4].text,
            "Points": span[5].text,
            "Plus-Minus": span[6].text,
            "Penalty-Mins": span[7].text,
            "Power-Play-Goals": span[8].text,
            "Power-Play-Points": span[9].text,
            "Short-Handed-Goals": span[10].text,
            "Short-Handed-Points": span[11].text,
            "Game-Winning-Goals": span[12].text,
            "Overtime-Goals": span[13].text,
            "Shots": span[14].text,
            "Shot-Perctange": span[15].text,
            "Face-Off-Win-Percent": span[16].text

        }
        return dict
    except:
        pass

__Creating a dataframe to store the data, then using a loop to populate data from specific years.__  
This loop takes a few minutes to complete because it has to load and run the scripts on each page to get the data.

In [4]:
draft_df = pd.DataFrame(columns=['Name', 'Team', 'Pick', 'Position', 'Country', 'Height', 'Weight', 'Year', 'Shoots', 'Games-Played', 'Goals', 'Assists', 'Points', 'Plus-Minus', 'Penalty-Mins', 'Power-Play-Goals', 'Power-Play-Points', 'Short-Handed-Goals', 'Short-Handed-Points', 'Game-Winning-Goals', 'Overtime-Goals', 'Shots', 'Shot-Percentage', 'Face-Off-Win-Percent'])
for year in range(1963, 2020):
    add_year_draft_data(year)
    time.sleep(0.5)

In [5]:
draft_df.to_csv('draft.csv')

In [6]:
draft_df

,Name,Team,Pick,Position,Country,Height,Weight,Year,Shoots,Games-Played,...,Power-Play-Goals,Power-Play-Points,Short-Handed-Goals,Short-Handed-Points,Game-Winning-Goals,Overtime-Goals,Shots,Shot-Percentage,Face-Off-Win-Percent,Shot-Perctange
0,Garry Monahan,Montréal Canadiens,1,LW,CAN,"6' 0""",199,1963,Left,748,...,15,33,6,14,17,0,"1,305",NaN,--,8.9
1,Peter Mahovlich,Detroit Red Wings,2,C,CAN,"6' 5""",210,1963,Left,883,...,72,202,19,25,38,0,"2,089",NaN,--,13.8
2,Orest Romashyna,Boston Bruins,3,LW,DEU,"5' 10""",170,1963,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Al Osborne,New York Rangers,4,RW,CAN,"6' 0""",180,1963,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Art Hampson,Chicago Blackhawks,5,D,CAN,,,1963,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290,Nolan Foote,Tampa Bay Lightning,27,LW,USA,"6' 3""",195,2019,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1291,Ryan Suzuki,Carolina Hurricanes,28,C,CAN,"6' 0""",180,2019,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1292,Brayden Tracey,Anaheim Ducks,29,LW,CAN,"6' 0""",170,2019,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1293,John Beecher,Boston Bruins,30,C,USA,"6' 3""",212,2019,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
print("Is the blank height value null?", draft_df.loc[4, 'Height'] == None)
print("It's datatype is:", type(draft_df.loc[4, 'Height']))

print("Is the blank Weight value null?", draft_df.loc[4, 'Weight'] == None)
print("It's datatype is:", type(draft_df.loc[4, 'Weight']))
#We see that our webscraper fills null values with str objects of length 0. 

Is the blank height value null? False
It's datatype is: <class 'str'>
Is the blank Weight value null? False
It's datatype is: <class 'str'>


In [8]:
draft_clean = draft_df[draft_df.Height != '']

In [9]:
#Now converting heights from foot and inches to inches